In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
host = 'localhost'         # 호스트 (예: '127.0.0.1')
user = 'root'              # 사용자 이름
password = '0000'      # 사용자 비밀번호
database = 'df_region'       # 데이터베이스 이름

# MySQL 연결 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

In [3]:
df_excel="excel_table"  # 불러올 테이블 이름
df_csv = 'csv_table'
df_csv2 = 'tourism_table'

In [4]:
df_review = pd.read_sql(df_csv, con=engine)
df_region2 = pd.read_sql(df_excel, con=engine)
df_tourism = pd.read_sql(df_csv2, con=engine)

In [5]:
import pandas as pd
from math import radians, sin, cos, sqrt, asin

# Haversine 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))

In [6]:
# df_trip_merged = 'trip_merged'

# df_tourism = pd.read_sql(df_trip_merged, con=engine)


In [7]:
# # 1. 자연열, 추천코스열 만들기
# df_tourism["자연"] = df_tourism["힐링"] / 2
# df_tourism["추천코스"] = df_tourism["힐링"] / 2

# # 2. 레포츠열 만들기
# df_tourism["레포츠"] = df_tourism["액티비티"]

# # 3. 인문열 만들기
# df_tourism["인문(문화/예술/역사)"] = df_tourism["문화"]

# df_tourism = df_tourism.drop(columns=["힐링", "액티비티", "문화"])

# # 3. 열 순서 재정렬
# first_cols = ["체험다양성_엔트로피", "체험다양성_표준편차", "위도", "경도"]
# other_cols = [col for col in df_tourism.columns if col not in first_cols]

# # 최종 열 순서
# new_cols = other_cols + first_cols

# # 열 순서 적용
# df_tourism = df_tourism[new_cols]

In [8]:
station_lat, station_lon = 37.5561, 126.9726

In [9]:
from math import radians, sin, cos, sqrt, asin
# Haversine 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))

In [10]:
# # 거리 계산 및 새로운 열 추가
# df_tourism["거리"] = df_tourism.apply(lambda row: haversine(station_lat, station_lon, row["위도"], row["경도"]), axis=1)

# # 5km 미만인 데이터 필터링
# df_filtered = df_tourism[df_tourism["거리"] < 5]

# # 거리순으로 정렬 후, 같은 거리 내에서 음식 점수 순으로 정렬
# df_sorted = df_filtered.sort_values(by=["자연", "거리"], ascending=[True, False])
# # 상위 10개 데이터 선택
# df_top10 = df_sorted.head(2)


In [11]:
# df_top10

In [12]:
# # 중복 확인용 세트
# recommended_foods = set()

# # 각 행정동별로 가까운 음식점 5개 찾기 (중복 제거)
# def find_top5_food(row):
#     lat, lon = row["위도"], row["경도"]
#     df_region2["거리"] = df_region2.apply(lambda r: haversine(lat, lon, r["mapy"], r["mapx"]), axis=1)
    
#     # 중복되지 않은 음식점 찾기
#     top_foods = []
#     for _, food in df_region2.sort_values("거리").iterrows():
#         if food["title"] not in recommended_foods:
#             top_foods.append(food["title"])
#             recommended_foods.add(food["title"])
#         if len(top_foods) == 5:  # 5개만 선택
#             break
#     return top_foods

# # 리스트로 저장
# closest_foods_list = [find_top5_food(row) for _, row in df_top10.iterrows()]

# closest_foods_list

In [13]:
# # 평탄화(Flatten)하여 한 행씩 추가
# flat_foods_list = [food for sublist in closest_foods_list for food in sublist]

# # 데이터프레임 생성
# df_recommend = pd.DataFrame({'추천장소': flat_foods_list})
# df_recommend

In [14]:
df_region  = pd.read_sql(df_excel, con=engine)
df_review  = pd.read_sql(df_csv, con=engine)

# ── 3) 리뷰 점수 계산 ───────────────────────────────────

# 3) addr1별 긍정점수 평균 계산
df_score = (
    df_review
    .groupby("addr1", as_index=False)["긍정점수"]
    .mean()
    # 컬럼명이 이미 "긍정점수"라면 rename은 생략해도 됩니다.
)

# 4) addr1별 리뷰내용 묶기
df_kw = (
    df_review
    .groupby("addr1", as_index=False)["리뷰내용"]
    .agg(lambda kws: 
        " ".join(
            sorted(
                set(
                    # None/NaN 제거
                    [k for k in kws if pd.notnull(k)]
                )
            )
        )
    )
)

# 5) df_region에 병합
df_region = (
    df_region
    .merge(df_score, on="addr1", how="left")
    .merge(df_kw,    on="addr1", how="left")
    .drop_duplicates(subset=["title"], keep="first")
    .reset_index(drop=True)
)

# 6) 리뷰내용 없을시 카테고리 cat3로 넣음
df_review['리뷰내용'] = df_review.apply(
lambda row: row['cat3'] if pd.isna(row['리뷰내용']) else row['리뷰내용'],
axis=1
)

In [15]:
df_region

,addr1,cat1,cat2,cat3,mapx,mapy,nearest_station,closest_subway_station,matched_station_name,정류장번호,line1_x,line2_x,line3_x,line4_x,title,긍정점수,리뷰내용
0,서울특별시 송파구 새말로 114 (문정동),음식,음식점,한식,127.126643,37.482681,문정법조단지.건영아파트(중),장지역,문정법조단지.건영아파트(중),GGB123000013,08호선,None,None,None,푼주,8.515263,가격만큼은 못했던거 같아요 그리고 물회가 많이 비렸는데 비리다고 말씀드렸더니 그럼 ...
1,서울특별시 성동구 무학로6길 22 (도선동),음식,음식점,카페/전통찻집,127.032527,37.565075,상왕십리역,상왕십리역,상왕십리역,GGB103000005,02호선,None,None,None,레이지 유스,7.590000,"분위기가 넘 조아요… 날씨 풀리면 테라스에서 시간 보내구 싶네요,,, 동네 주택가에..."
2,서울특별시 강서구 마곡중앙로 161-8 (마곡동),음식,음식점,이색음식점,126.827126,37.568982,마곡나루역1번출구,마곡나루역,마곡나루역1번출구,ICB115000898,09호선,공항철도,None,None,105405 마곡,8.696316,Lunch Course6가지 음식이 나오네요. 빵 부터 디져트까지.직접 만든 양념과...
3,서울특별시 성북구 동소문로20다길 30 (동선동1가),음식,음식점,카페/전통찻집,127.018306,37.592083,돈암사거리.성신여대입구(중),성신여대입구역,돈암사거리.성신여대입구(중),GGB107000007,04호선,우이신설경전철,None,None,서울 본크레페,8.282647,5년째 가는 단골♡♡오후 두시부터 열고 솔드아웃되면 닫으시는데 보통 5-6시?? 자...
4,서울특별시 강북구 도봉로68길 36 (미아동),음식,음식점,카페/전통찻집,127.026445,37.631395,수유시장.성신여대미아캠퍼스앞(중),미아역,수유시장.성신여대미아캠퍼스앞(중),GGB108000005,04호선,None,None,None,리코,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13965,경기도 화성시 효행로1265번길 76,음식,음식점,카페/전통찻집,127.055138,37.225603,반월중.고등학교앞,망포역,반월중.고등학교앞,GGB233002502,분당선,None,None,None,카페반월,7.652500,맛있고 가게도 깔끔한데 손님관리좀 하시면 좋겠어요동네장사인건 이해하지만 지나치게 시...
13966,경기도 화성시 효행로1374번길 24,음식,음식점,카페/전통찻집,127.067525,37.229089,신동탄롯데캐슬,망포역,신동탄롯데캐슬,GGB233002000,분당선,None,None,None,브라운스케이프,7.956190,My friend and I went to this cafe today. We ar...
13967,경기도 화성시 효행로481번길 21 (안녕동),인문(문화/예술/역사),역사관광지,유적지/사적지,126.989286,37.208185,융건릉입구,병점역,융건릉입구,GGB233001840,01호선,None,None,None,화성 융건릉,NaN,NaN
13968,경기도 화성시 효행로481번길 21 (안녕동),인문(문화/예술/역사),역사관광지,유적지/사적지,126.989286,37.208185,융건릉입구,병점역,융건릉입구,GGB233001840,01호선,None,None,None,화성 융릉(장조·헌경황후)과 건릉(정조·효의황후) [유네스코 세계유산],NaN,NaN


In [16]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# df_region에서 '음식' 카테고리만 추출
df_food = df_region[df_region['cat1'] == '음식']

# 긍정점수 컬럼을 숫자형으로 변환
df_food['긍정점수'] = pd.to_numeric(df_food['긍정점수'], errors='coerce')

# 긍정점수가 없는 데이터는 제외
df_food = df_food.dropna(subset=['긍정점수'])

# 긍정점수 기준으로 정렬
df_food_sorted_by_score = df_food.sort_values(by='긍정점수', ascending=False)

# NearestNeighbors 모델 설정 (위도, 경도를 기준으로 추천)
neighbors_model = NearestNeighbors(n_neighbors=5, radius=5, algorithm='ball_tree')
neighbors_model.fit(df_food_sorted_by_score[['mapy', 'mapx']])

# 중복된 추천을 방지하기 위한 집합
recommended_foods = set()

# 음식점 추천 함수 (중복 없이 5개 추천)
def find_nearby_foods(place_name):
    place_row = df_food_sorted_by_score[df_food_sorted_by_score['title'] == place_name]

    if place_row.empty:
        return [], 0  # 해당 음식점이 없으면 빈 리스트 반환

    lat, lon = place_row.iloc[0]['mapy'], place_row.iloc[0]['mapx']
    distances, indices = neighbors_model.radius_neighbors([[lat, lon]], radius=5)

    nearby_foods = []
    walk_distances = []

    for idx, dist in zip(indices[0], distances[0]):
        food_name = df_food_sorted_by_score.iloc[idx]['title']
        if food_name not in recommended_foods:
            nearby_foods.append(food_name)
            walk_distances.append(dist)
            recommended_foods.add(food_name)
        if len(nearby_foods) == 5:  # 최대 5개만 추천
            break

    avg_walk_distance = round(sum(walk_distances) / len(walk_distances), 2) if walk_distances else 0
    return nearby_foods, avg_walk_distance

# 추천 데이터프레임 준비 (상위 10개의 음식점 추천)
df_recommend = pd.DataFrame({
    "추천장소": df_food_sorted_by_score["title"].head(10)  # 긍정점수 기준 상위 10개 음식점
})

# 첫 번째 추천장소에서 가까운 음식점 5개 및 도보 이동 거리 추가
df_recommend["추천장소2"], df_recommend["도보이동km_1"] = zip(*df_recommend["추천장소"].apply(find_nearby_foods))

# 추천장소2에서 가까운 음식점 5개 및 도보 이동 거리 추가 (추천장소2를 기준으로)
df_recommend["추천장소3"], df_recommend["도보이동km_2"] = zip(*df_recommend["추천장소2"].apply(lambda places: find_nearby_foods(places[0]) if places else ([], 0)))

# 🔹 지정된 열 순서대로 정렬
df_recommend = df_recommend[["추천장소", "도보이동km_1", "추천장소2", "도보이동km_2", "추천장소3"]]

# 리스트를 문자열로 변환
df_recommend["추천장소2"] = df_recommend["추천장소2"].apply(lambda x: ", ".join(x) if isinstance(x, list) else str(x))
df_recommend["추천장소3"] = df_recommend["추천장소3"].apply(lambda x: ", ".join(map(str, x)) if isinstance(x, list) else str(x))

# 결과 확인
df_recommend


C:\Users\hyunj\AppData\Local\Temp\ipykernel_9868\3732445950.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_food['긍정점수'] = pd.to_numeric(df_food['긍정점수'], errors='coerce')


,추천장소,도보이동km_1,추천장소2,도보이동km_2,추천장소3
8350,우나기강,0.70,"동해막국수, 고목정쌈밥, 철이네물회전문점, 해송쌈밥, 카페,잠진도길28",0.19,"공항샤브칼국수, 서해수산, 조은전골칼국수, 만정수산, 대부객주"
5711,가평 다하랑,1.10,"바다드림, C27 다운타운, 현미네 조개구이, 엠클리프, 늘목",0.20,"빈솔트, 엔틸로프, 와각칼국수, 서당골막국수, 조만간식당"
6829,하루방가든,0.81,"우리밀손칼국수, 을왕리빵판다카페, 을왕리짱구네, 미애네칼국수, 미송쌈정식",0.20,"아라짬뽕, 파도마루, 충무공이순신횟집, 바닷마을떡볶이, 유가네 칼국수"
12853,함흥집,0.75,"양자강, 스타파이브 카페, 서해안조개광장 신관, 불티나조개구이, 공항마을",0.28,"배터지는집, 대부도손칼국수, 이영화교동짬뽕 본점, 리베르테, 양평해장국 고흥금산매생이국밥"
1368,청춘극장,0.63,"기와집담, 어부신광호, 마시안어부집, 솔밭회조개구이, 무의도데침쌈밥",0.21,"옛날보리밥삼색칼국수, 33호할머니칼국수, 포도밭할머니손칼국수전문점, 삐죽이칼국수, ..."
9140,김가네제면소,0.35,"해상궁, 동양염전베이커리카페, 거해짬뽕순두부, 카페라봉, 을왕리300도씨 해물갈비",0.26,"왕할머니손칼국수, 16호원조할머니칼국수, 11호남도할머니칼국수, 대부옥수수찐빵, 장..."
1896,차돌박힌쭈꾸미 종로,0.64,"섬마을다리집, 회나라식당, 코세드, 용경향, 하늘채까페",0.29,"황금손칼국수 간장게장, 북동삼거리손칼국수, 대갓집 해물왕창손칼국수&직화쭈꾸미, 엄지..."
10281,본가양평해장국 양평지점,1.00,"플로레도커피, 고양이역 카페, 하이바다, 명품관, 영흥도바지락해물칼국수",0.11,"신우정, 불난그집, 카페제부리, 까망조개이야기, 문성5호"
12913,현씨주방,0.69,"본토칼국수, 태평양호, 체이스커피, 바람의마을, 뻘다방",0.16,"솔밭칼국수, 롱비치커피앤브래드, 황성얼큰오징어찌개, 26호까치할머니손칼국수, 임파스토"
6753,현촌,0.69,"카페폰테, 지금이곳, 쌍곰다방, 선재리커피집, 영종원조국수",0.15,"기노스코, 할머니소머리국밥&속초코다리냉면, 진짜원조소나무집, 연안부두 목포신안18호..."
